In [6]:
import matplotlib
import math
from pylab import *
import numpy as np
import itertools
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import os
import imageio
import random
import statistics as st
import scipy.stats as ss
import matplotlib.patches as patches

#Parámetros
N = [5,10,15,20,25,30,35,40,45] #cantidad de agentes por experimento (densidad).
pt = 1 #paso temporal
s = 100 #cantidad de simulaciones por densidad
t = 500 #tiempo que dura cada simulación
vmax = 1 #máxima velocidad que puede tener un agente
vmin= 0 #mínima velocidad
tM= 50 #capacidad máxima de agentes por carril

class Vehiculo:
    id_iter = itertools.count() #para crearles un id a los agentes
    def __init__(self,tipo,posx,velocidad,posy):
        self.id = next(Vehiculo.id_iter) #id
        self.t = tipo #para poder duplicar agentes y simular el espacio toroidal
        self.x = posx #para la posición del vehículo, coordenada horizontal
        self.y = posy #para la posición del vehículo, coordenada vertical
        self.vm = velocidad #velocidad del vehículo en el paso
        self.d = 0 #distancia recorrida por paso
        self.a = 0.01 #valor default de la aceleración
        self.m = "vmax" #etiqueta del valor considerado para comportamiento en la distancia de seguimiento
        self.f = vmax  #valor default considerado para comportamiento en la distancia de seguimiento.
        self.inc="No" #etiqueta para identificar a los transgersores que desaceleran sin sentido.
        self.compor="Responsable" #valor default de los agentes, se puede variar a indiferente, oportunista y transgresor

    def mover(self,acel): #función para la actualización de la distancia recorrida por un agente
        self.a = acel
        self.vm = max(vmin, min(vmax, self.vm + self.a*pt))
        self.d =self.vm*pt


def iniciar(n): #con esta función se crean los agentes del modelo
  global vehiculos, filenames
  filenames = [] #lista para guardar las imágenes del GIF de la simulación
  vehiculos = [] #lista para guardar todos los agentes tipo vehículo.

  while len(vehiculos) < n*0.25: #para crear agentes con comportamiento oportunista
      v = Vehiculo("carro", round(np.random.uniform(0,tM),0), round(np.random.uniform(0.1,0.3),4),1)
      v.compor="Oportunista"
      v.m="vm"
      if round(v.x,0) not in [round(z.x,0) for z in vehiculos]:
          vehiculos.append(v)

  while len(vehiculos) < n*0.5: #para crear agentes con comportamiento indiferente
      v = Vehiculo("carro", round(np.random.uniform(0,tM),0), round(np.random.uniform(0.1,0.3),4),1)
      v.compor="Indiferente"
      v.m=random.choice(["vm","vmax"])
      if round(v.x,0) not in [round(z.x,0) for z in vehiculos]:
          vehiculos.append(v)

  while len(vehiculos) < n*0.75: #para crear agentes con comportamiento transgresor
      v = Vehiculo("carro", round(np.random.uniform(0,tM),0), round(np.random.uniform(0.1,0.3),4),1)
      v.compor="Transgresor"
      v.m=random.choice(["vm","vmax"])
      if round(v.x,0) not in [round(z.x,0) for z in vehiculos]:
          vehiculos.append(v)

  while len(vehiculos) < n: #para crear agentes con comportamiento responsable
      v = Vehiculo("carro", round(np.random.uniform(0,tM),0), round(np.random.uniform(0.1,0.3),4),1)
      if round(v.x,0) not in [round(z.x,0) for z in vehiculos]:
          vehiculos.append(v)


def toroide(): #para simular un espacio toroidal
    for v in vehiculos:
      if v.x > tM:
          v.x = v.x % tM

def duplicar(): #función para duplicar al agente que está al inicio del carril y ubicarlo adelante del que está al último para que el último sepa cómo moverse.
    global a
    for v in vehiculos:
      if v.x == min([v.x for v in vehiculos]):
        a = v.vm
        copia = Vehiculo("doble",min([v.x for v in vehiculos])+tM, a,1)
        vehiculos.append(copia) #se añade el vehículo copiado temporalmente

def parametros(v,vm): #función para que cada carro actualice su parámetro de comportamiento en base a la etiqueta que se le asignó en la función iniciar()
  distancias = {
      ("vmax"): vmax, #distancia de seguimiento fija
      ("vm"): vm} #distancias de seguimiento variable
  return distancias.get((v),"NA")

def imitacion(): #función para el comportamiento de imitación de la distancia de seguimiento, solo en indiferentes
  alrededor=[] #lista para guardar a los vehículos que están alrededor del evaluado
  for v in vehiculos:
    for z in vehiculos:
      if v!=z:
        if abs(v.x - z.x) <= 1.1:
          alrededor.append(z.m)
    if len(alrededor)!=0:
      if len([i for i in alrededor if i == "vmax"])>len([i for i in alrededor if i == "vm"]) and v.compor=="Indiferente":
        v.m="vmax"
      elif len([i for i in alrededor if i == "vmax"])<len([i for i in alrededor if i == "vm"]) and v.compor=="Indiferente":
        v.m="vm"
      else:
        pass
    alrededor=[]

def movimiento(): #función para que cada agente tipo vehículo sepa cómo moverse.
    vehiculos.sort(key = lambda v: v.x)
    for v,z in zip(vehiculos,vehiculos[1:]):
        d = z.x - v.x #distancia entre vehículos
        v.f = parametros(v.m, min(1,max(v.vm + 0.011,0.1))) #actualización del valor considerado para el comportamiento en la distancia de seguimiento.
        if  d >= v.f*pt: #si la distancia entre vehículos es mayor al valor de la distancia de seguimiento, acelera.
            v.mover(0.01+0.001*random.uniform(-1,1)) #aumento de velocidad.
        else: #si es menor, desacelera
            v.vm = z.vm #primero, iguala la velocidad del vehículo de adelanteo
            v.mover(-0.02) # y luego reduce su velocidad.

        if random.random()<0.1 and v.compor=="Transgresor": #adicionalmente existe un 10% de probabilidad de que un transgresor desacelere sin sentido
          v.inc="Si"
          v.mover(-0.02)

        v.x = v.d + v.x #se actualiza la nueva posición del vehículo en base a la distancia recorrida.

def borrar(): #función para borrar el duplicado creado
    for v in vehiculos:
        if v.t == "doble":
            vehiculos.remove(v)


def interaccion(n,s): #función que une las funciones que se necesitan en cada paso de la simulación
    global filenames
    for i in range(1,t+1): #se ejecuta el loop la cantidad de pasos (duración) que tiene cada simulación
        duplicar()
        imitacion()
        movimiento()
        borrar()
        toroide()


        #Información recopilada por simulación para generar gráficos. Se puede recopilar otro tipo de información en base a lo requerido.
        vel1.append([n,st.mean([v.vm for v in vehiculos]),s])
        med1.append([n,st.median([v.vm for v in vehiculos]),s])
        des1.append([n,st.stdev([v.vm for v in vehiculos]),s])
        err1.append([n,ss.sem([v.vm for v in vehiculos]),s])

        #Código para generar una imagen de cada paso por simulación y poder generar el GIF de toda la simulación. Para hacer el GIF, se descomenta lo que viene a continuación:
        '''
        ig, ax = plt.subplots(figsize=(18,7))
        #carros
        #plot([v.x for v in vehiculos if v.m=="vmax"], [v.y for v in vehiculos if v.m=="vmax"] ,'o', color="darkorange", markersize=3)
        #plot([v.x for v in vehiculos if v.m=="vm"], [v.y for v in vehiculos if v.m=="vm"] ,'o', color="blue", markersize=3)
        #plot([v.x for v in vehiculos if v.inc=="Si"], [v.y for v in vehiculos if v.inc=="Si"] ,'o', color="yellow", markersize=3)
        plot([v.x for v in vehiculos if v.compor=="Oportunista"], [v.y for v in vehiculos if v.compor=="Oportunista"] ,'o', color="blue", markersize=3)
        plot([v.x for v in vehiculos if v.compor=="Indiferente"], [v.y for v in vehiculos if v.compor=="Indiferente"] ,'o', color="red", markersize=3)
        plot([v.x for v in vehiculos if v.compor=="Responsable"], [v.y for v in vehiculos if v.compor=="Responsable"] ,'o', color="green", markersize=3)
        plot([v.x for v in vehiculos if v.compor=="Transgresor" and v.inc=="Si"], [v.y for v in vehiculos if v.compor=="Transgresor" and v.inc=="Si"] ,'o', color="black", markersize=3)
        plot([v.x for v in vehiculos if v.compor=="Transgresor" and v.inc=="No"], [v.y for v in vehiculos if v.compor=="Transgresor" and v.inc=="No"] ,'o', color="white", markersize=3)
        #carriles
        ax.add_patch(patches.Rectangle((0, 0.8),50,0.4,edgecolor = 'gray',facecolor = 'gray',fill=True))
        text(1,0.17,"Paso:")
        text(2,0.2,i)
        tick_params(labelcolor='w')
        #grid(b=True, axis='both', color = "white")
        axis([0,tM,0.5,1.5])

        filename = f'paso{i}.png'
        filenames.append(filename)
        plt.savefig(filename)
        plt.close()
        #'''
        '''
        print(["Tiempo:",i])
        print(len([v for v in vehiculos if v.m=="vmax"]))
        print(len([v for v in vehiculos if v.m=="vm"]))
        '''
        for v in vehiculos: #esto está acá para que se pueda pintar en el GIF a los transgresores que desaceleran sin sentido.
          v.inc="No"


In [7]:
#Este bloque ejecuta el modelo
#listas para guardar la información de los gráficos
vel1 = []
med1 = []
des1 = []
err1 = []

for i in N: #para cada cantidad de carros (i) por simulación, se ejecuta (j) cantidad de simulaciones.
  for j in range (1,s+1): #el s+1 es la cantidad de simulaciones
    iniciar(i)
    interaccion(i,j)
  media=[]
  media.append(st.mean([l[1] for l in vel1 if l[0]==i]))
  print(media)
  #print(["T:",len([v for v in vehiculos if v.compor=="Transgresor"]),"O:",len([v for v in vehiculos if v.compor=="Oportunista"]),"I:",len([v for v in vehiculos if v.compor=="Indiferente"]),"R:",len([v for v in vehiculos if v.compor=="Responsable"])],)
  print("-----------------------------","Acabó N =",i,"-----------------------------")


[0.9287433243238634]
----------------------------- Acabó N = 5 -----------------------------
[0.9183436816313648]
----------------------------- Acabó N = 10 -----------------------------
[0.9141294630917077]
----------------------------- Acabó N = 15 -----------------------------
[0.9050808052125794]
----------------------------- Acabó N = 20 -----------------------------
[0.8874854413185725]
----------------------------- Acabó N = 25 -----------------------------
[0.591618570109146]
----------------------------- Acabó N = 30 -----------------------------
[0.4049349914178996]
----------------------------- Acabó N = 35 -----------------------------
[0.3197447356097451]
----------------------------- Acabó N = 40 -----------------------------
[0.2661338588472976]
----------------------------- Acabó N = 45 -----------------------------


In [ ]:
#Código para generar el GIF, siempre y cuando se hayan guardado las imágenes por paso
gif_name= "Los 4 comportamientos"

with imageio.get_writer(f'{gif_name}.gif', mode='I') as writer:
    for filename in filenames:
        i = imageio.imread(filename)
        writer.append_data(i)

#Esto elimina automáticamente las imágenes por paso creadas
for filename in set(filenames):
    os.remove(filename)

<ipython-input-27-a2be6a0d78e3>:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  i = imageio.imread(filename)
